In [63]:
import requests, json
from dotenv import dotenv_values
corcelKey = dotenv_values(".env")['corcel_apikey']


url = "https://api.corcel.io/vision/embed_text"

payload = {
    "text_prompts": ["bicycle"]
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": corcelKey
}

response = requests.post(url, json=payload, headers=headers)

jsonresponse = json.loads(response.text)
text_embedding = jsonresponse[0]['text_embeddings'][0]

#print(text_embedding)



In [64]:
import requests
import base64
image_urls = [
    "https://storage.googleapis.com/corcel-images/d36a1306-6973-4d94-9.webp",
    "https://storage.googleapis.com/corcel-images/8da56a62-6129-48c7-b.webp",
    "https://storage.googleapis.com/corcel-images/1dfa33d5-f1e9-42c9-a.webp",
    "https://storage.googleapis.com/corcel-images/74e45f262c5f5748954b617cc2fbc103.webp",
    "https://storage.googleapis.com/corcel-images/2df9fa91-7fac-4dd2-a.webp",
    "https://storage.googleapis.com/corcel-images/6debe5d9-a9ba-45eb-b.webp"
]

b64_images = []

for image in image_urls:
    response = requests.get(image)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Convert the image content to base64
        base64_image = base64.b64encode(response.content).decode('utf-8')
    else:
        base64_image = ""
    
    b64_images.append(base64_image)



In [65]:
#encode the B64 images into vectors

url = "https://api.corcel.io/vision/embed_image"

payload = {
    "image_b64s": b64_images
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": corcelKey
}

response = requests.post(url, json=payload, headers=headers)


jsonresponse = json.loads(response.text)
image_embedding = jsonresponse[0]['image_embeddings']

print(len(image_embedding))


6


In [66]:
import numpy as np
from IPython.display import display, Image

#create the text vector
textVector = np.array(text_embedding)

#store dot products in ana array
dotProducts = []

#loop through the images, create the vector, do a dot product, and place the solution in the dotProducts array
for imageVector in image_embedding:

    dotProd = np.dot(textVector, imageVector)
    dotProducts.append(dotProd)

#sort the dot products highest to lowest
sorted_indices = np.argsort(dotProducts)[::-1]

#from highest to lowest, show the dot product (with 3 digits after the decimal), and the image.
for index in sorted_indices:
    print(f"similarity : {dotProducts[index]:.3f}")
    display(Image(url=image_urls[index], width =400))

similarity : 32.296


similarity : 30.858


similarity : 30.001


similarity : 29.718


similarity : 25.337


similarity : 23.471
